In [1]:
import pandas as pd
import matplotlib.pylab as plt
import numpy as np
import seaborn as sns

from tqdm.auto import tqdm

tqdm.pandas()
%matplotlib inline
pd.set_option("display.max_rows", 1000)

/home/rexhaif/Projects/RussianObnulenie2020/.venv/lib/python3.7/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
data = pd.read_csv("uiks-utf8.csv")

In [3]:
data.head()

,region_name,tik_name,uik_name,registered_voters,used_ballots,found_ballots,incorrect_ballots,yes_votes,no_votes,latitude,longitude
0,Республика Адыгея (Адыгея),1 Адыгейская,УИК №1,2277,1925,1925,0,1749,176,NaN,NaN
1,Республика Адыгея (Адыгея),1 Адыгейская,УИК №2,2694,2290,2290,7,2117,166,44.983019,39.103872
2,Республика Адыгея (Адыгея),1 Адыгейская,УИК №3,2865,2441,2441,11,2171,259,NaN,NaN
3,Республика Адыгея (Адыгея),1 Адыгейская,УИК №4,2054,1669,1669,8,1381,280,44.887695,39.202791
4,Республика Адыгея (Адыгея),1 Адыгейская,УИК №5,714,607,607,1,473,133,44.883646,39.183058


## Сумма голосов за/против

In [4]:
print(f"{data.yes_votes.sum()} Голосов За")
print(f"{data.no_votes.sum()} Голосов Против")

57071119 Голосов За
15347937 Голосов Против


## Ловушка удобных чисел

In [15]:
lucky_divider = 0.5

data['pct_turnout'] = (data.found_ballots / data.registered_voters) * 100
data['pct_yes']     = (data.yes_votes / data.found_ballots) * 100

data['lucky_turnout'] = data['pct_turnout'] % lucky_divider == 0
data['lucky_yes'] = data['pct_yes'] % lucky_divider == 0

data['lucky_all'] = data.progress_apply(lambda x: x.lucky_turnout & x.lucky_yes, axis=1)
data['lucky_any'] = data.progress_apply(lambda x: x.lucky_turnout | x.lucky_yes, axis=1)

In [16]:
## Удалим из выборки потенциальные участки на предприятиях, военных частях и больницах, с небольшим числом зарегистрированных избирателей
## Как правило на таких участках 100% явка и это относительно нормально, и не свидетельствует о фальсификациях
data_ = data[data.registered_voters > 500]
print(f"Подвыборка из {(len(data_) / len(data)) * 100:.2f}% участков")

Подвыборка из 67.25% участков


### Явка

In [17]:
print("Самые часто-встречащиеся проценты явки")
print("-"*40)
turnout_counts = data_['pct_turnout'].value_counts()
for key, val in list(turnout_counts.iteritems())[:20]:
    print(f"{key:^20} | {val:^10}")

Самые часто-встречащиеся проценты явки
----------------------------------------
       100.0         |    212    
        75.0         |     57    
        90.0         |     39    
        50.0         |     33    
        80.0         |     31    
        95.0         |     25    
 85.71428571428571   |     25    
 66.66666666666666   |     20    
  90.9090909090909   |     19    
        40.0         |     17    
        87.5         |     16    
 83.33333333333334   |     14    
 71.42857142857143   |     14    
        70.0         |     14    
  92.3076923076923   |     14    
 77.77777777777779   |     12    
        60.0         |     12    
 91.66666666666666   |     12    
 80.95238095238095   |     11    
 86.20689655172413   |     11    


In [18]:
print(f"{len(data_[data_['pct_turnout'] % lucky_divider == 0])} участков, на которых процент явки нацело делится на {lucky_divider}")

612 участков, на которых процент явки нацело делится на 0.5


### Голоса ЗА поправки

In [19]:
print("Самые часто-встречащиеся проценты голосов ЗА поправки")
print("-"*40)
yes_counts = data_['pct_yes'].value_counts()
for key, val in list(turnout_counts.iteritems())[:20]:
    print(f"{key:^20} | {val:^10}")

Самые часто-встречащиеся проценты голосов ЗА поправки
----------------------------------------
       100.0         |    212    
        75.0         |     57    
        90.0         |     39    
        50.0         |     33    
        80.0         |     31    
        95.0         |     25    
 85.71428571428571   |     25    
 66.66666666666666   |     20    
  90.9090909090909   |     19    
        40.0         |     17    
        87.5         |     16    
 83.33333333333334   |     14    
 71.42857142857143   |     14    
        70.0         |     14    
  92.3076923076923   |     14    
 77.77777777777779   |     12    
        60.0         |     12    
 91.66666666666666   |     12    
 80.95238095238095   |     11    
 86.20689655172413   |     11    


In [20]:
print(f"{len(data_[data_['pct_yes'] % lucky_divider == 0])} участков, на которых процент голосов ЗА нацело делится на {lucky_divider}")

713 участков, на которых процент голосов ЗА нацело делится на 0.5


### Что чаще рисуют, голоса ЗА или явку(или и то и то)?

In [21]:
data_['lucky_turnout'].value_counts()

False    64463
True       612
Name: lucky_turnout, dtype: int64

In [22]:
data_['lucky_yes'].value_counts()

False    64362
True       713
Name: lucky_yes, dtype: int64

In [23]:
data_['lucky_all'].value_counts()

False    65066
True         9
Name: lucky_all, dtype: int64

Чаще рисуют явку, чуть реже голоса ЗА, но почти никогда не рисуют и то и другое одновременно

## Какие регионы попали в ловушку удобных чисел

In [28]:
data_[data_['lucky_any']]['region_name'].value_counts()

Московская область                          73
Краснодарский край                          67
город Москва                                58
Территория за пределами РФ                  45
город Санкт-Петербург                       42
Нижегородская область                       35
Ставропольский край                         35
Республика Башкортостан                     35
Саратовская область                         34
Республика Дагестан                         34
Ростовская область                          34
Свердловская область                        32
Пермский край                               32
Воронежская область                         30
Республика Татарстан (Татарстан)            30
Волгоградская область                       28
Челябинская область                         27
Ленинградская область                       24
Иркутская область                           24
Новосибирская область                       23
Брянская область                            20
Кемеровская о

## Суммарно больше рисовали красивые числа в Москве и области, Питере, Краснодарском крае. Возможно рисовали и на зарубежных участках, но хз, они сами по себе странные.

In [29]:
data_[data_['lucky_all']]

,region_name,tik_name,uik_name,registered_voters,used_ballots,found_ballots,incorrect_ballots,yes_votes,no_votes,latitude,longitude,pct_turnout,pct_yes,lucky_turnout,lucky_yes,lucky_all,lucky_any
2651,Республика Башкортостан,49 Стерлитамакская городская,УИК №683,1888,1652,1652,2,1239,411,53.652513,55.951040,87.5,75.0,True,True,True,True
3011,Республика Башкортостан,"55 Уфа, Кировская",УИК №164,604,604,604,0,604,0,NaN,NaN,100.0,100.0,True,True,True,True
3330,Республика Башкортостан,61 Учалинская,УИК №3093,2000,1900,1900,0,1805,95,54.207878,59.598730,95.0,95.0,True,True,True,True
55160,Московская область,36 Люберецкая городская,УИК №1463,1380,1035,1035,5,828,202,55.669821,37.878534,75.0,80.0,True,True,True,True
58764,Нижегородская область,13 Ветлужская,УИК №522,675,540,540,0,432,108,58.000555,45.125448,80.0,80.0,True,True,True,True
65235,Оренбургская область,6 Беляевская,УИК №197,625,500,500,5,405,90,51.368448,55.799254,80.0,81.0,True,True,True,True
76390,Саратовская область,46 Энгельсская,УИК №1798,2500,1850,1850,0,1739,111,51.468896,46.146375,74.0,94.0,True,True,True,True
89605,город Москва,4 район Гольяново,УИК №986,1940,1455,1455,3,1164,288,55.818680,37.828892,75.0,80.0,True,True,True,True
92000,город Москва,100 Южнопортовый район,УИК №1644,1760,1100,1100,0,902,198,55.706482,37.672190,62.5,82.0,True,True,True,True


In [30]:
data['lucky_numbers'] = data['lucky_any'].astype(int)

In [31]:
data = data.drop(['lucky_yes', 'lucky_turnout', 'lucky_all', 'lucky_any'], axis=1)

## Теперь дадим избиркомам право ошибится на 1-2 бюллетеня при рисовке, посмотрим на проценты явки\голосов. Близкие к удобным числам

In [50]:
tol = 1e-2
data['close_lucky_turnout'] = np.isclose(data['pct_turnout'] % lucky_divider, [0] * len(data), atol=tol)
data['close_lucky_yes']     = np.isclose(data['pct_yes'] % lucky_divider, [0] * len(data), atol=tol)

data['close_lucky_any']     = data.apply(lambda x: x.close_lucky_turnout or x.close_lucky_yes, axis=1)
data['close_lucky_all']     = data.apply(lambda x: x.close_lucky_turnout and x.close_lucky_yes, axis=1)

In [51]:
data_ = data[data.registered_voters > 500]

In [52]:
print(f"{len(data_[data_['close_lucky_turnout']])} участков, на которых процент явки нацело делится на {lucky_divider} с точностью до {tol}")

598 участков, на которых процент явки нацело делится на 5.0 с точностью до 0.01


In [53]:
print(f"{len(data_[data_['close_lucky_yes']])} участков, на которых процент голосов ЗА нацело делится на {lucky_divider} c точностью до {tol}")

538 участков, на которых процент голосов ЗА нацело делится на 5.0 c точностью до 0.01


In [54]:
print(f"{len(data_[data_['close_lucky_any']])} участков всего с подозрением на нарушения такого рода")

1126 участков всего с подозрением на нарушения такого рода


In [55]:
print(f"{len(data_[data_['close_lucky_all']])} участков нарисовали и явку и процент голосов одновременно")

10 участков нарисовали и явку и процент голосов одновременно


In [56]:
data_[data_['close_lucky_all']]

,region_name,tik_name,uik_name,registered_voters,used_ballots,found_ballots,incorrect_ballots,yes_votes,no_votes,latitude,longitude,pct_turnout,pct_yes,lucky_numbers,close_lucky_turnout,close_lucky_yes,close_lucky_any,close_lucky_all
3011,Республика Башкортостан,"55 Уфа, Кировская",УИК №164,604,604,604,0,604,0,NaN,NaN,100.000000,100.000000,1,True,True,True,True
3330,Республика Башкортостан,61 Учалинская,УИК №3093,2000,1900,1900,0,1805,95,54.207878,59.598730,95.000000,95.000000,1,True,True,True,True
20229,Алтайский край,62 Топчихинская,УИК №1631,1057,370,370,1,259,110,52.782948,83.032834,35.004730,70.000000,1,True,True,True,True
24156,Краснодарский край,51 Тихорецкая районная,УИК №5108,1222,1161,1161,0,1103,58,45.671213,40.281260,95.008183,95.004307,0,True,True,True,True
31260,Ставропольский край,"31 Ставрополь, Промышленная",УИК №126,1122,1122,1122,6,1066,50,NaN,NaN,100.000000,95.008913,1,True,True,True,True
36371,Брянская область,18 Клинцовская городская,УИК №547,1890,1701,1701,90,1446,165,52.756184,32.255738,90.000000,85.008818,1,True,True,True,True
55160,Московская область,36 Люберецкая городская,УИК №1463,1380,1035,1035,5,828,202,55.669821,37.878534,75.000000,80.000000,1,True,True,True,True
58764,Нижегородская область,13 Ветлужская,УИК №522,675,540,540,0,432,108,58.000555,45.125448,80.000000,80.000000,1,True,True,True,True
89605,город Москва,4 район Гольяново,УИК №986,1940,1455,1455,3,1164,288,55.818680,37.828892,75.000000,80.000000,1,True,True,True,True
94255,город Санкт-Петербург,18 Территориальная избирательная комиссия №18,УИК №1653,2535,2155,2155,18,1724,413,NaN,NaN,85.009862,80.000000,1,True,True,True,True


### Отличившиеся регионы

In [57]:
data_[data_['close_lucky_any']]['region_name'].value_counts()

Московская область                          68
Краснодарский край                          55
город Москва                                48
город Санкт-Петербург                       47
Республика Башкортостан                     46
Территория за пределами РФ                  44
Ставропольский край                         43
Нижегородская область                       34
Ростовская область                          34
Республика Дагестан                         33
Воронежская область                         27
Волгоградская область                       27
Брянская область                            25
Республика Татарстан (Татарстан)            25
Свердловская область                        22
Саратовская область                         22
Пермский край                               20
Приморский край                             20
Ленинградская область                       19
Челябинская область                         19
Кемеровская область - Кузбасс               19
Красноярский 